In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import random
import warnings
from time import time
warnings.filterwarnings('ignore')

In [2]:
def create_graph():
    
    x=[0.09,0.16,0.84,0.70]
    y=[0.17,0.52,0.92,0.16]
    
    n=len(x)
    
    l=np.zeros((n,n))
    
#     print(l)
    
    for i in range(n):
        for j in range(n):
            x1=x[i]
            y1=y[i]
            x2=x[j]
            y2=y[j]
            
            ecd_dist= np.sqrt( (x1-x2)**2 + (y1-y2)**2 )
            
            l[i,j]=ecd_dist
            
            
    print(l)
    
    return l

In [3]:
graph=create_graph()
type(graph)

[[0.         0.35693137 1.06066017 0.61008196]
 [0.35693137 0.         0.78892332 0.64899923]
 [1.06066017 0.78892332 0.         0.77278716]
 [0.61008196 0.64899923 0.77278716 0.        ]]


numpy.ndarray

In [4]:
## rolette_wheel function to choose the best node based on probability

In [5]:

def rolette_wheel(p):

    cumsum = np.cumsum(p)

    r = random.random()

    next_node = np.where(r <= cumsum)

    next_node = next_node[0][0]

    return next_node

In [6]:
## create colony function

In [7]:
def create_colony(graph, ant_no, tau, eta, alpha, beta):
    n = len(graph)
    
#     print(n)

    colony = []

    for i in range(ant_no):
        tour = []
        initial_node = random.randint(0, n - 1)  # select a random node

        tour.append(initial_node)

        P_allNodes = tau ** alpha * eta ** beta

        for j in range(1, n):  # choose the rest of nodes

            currentNode = tour[-1]

            p = P_allNodes[initial_node][:]

            p[tour[-1]] = 0  # assign 0 to already visited node

            P = p / np.sum(p)

            next_node = rolette_wheel(P)
            
            tour.append(next_node)

        tour.append(tour[0])

        colony.append(tour)
    

    return colony

In [8]:
## to calc the fitness of all the ant ( i.e. cost/ distance)

In [9]:
def fitness(tour, graph):
    fitness = 0

    for i in range(len(tour) - 1):
        current_node = tour[i]
        next_node = tour[i + 1]

        fitness = fitness + graph[current_node][next_node]

    return fitness

In [10]:
## function to update_phromone  

In [11]:
def local_update_phromone(tau, colony, antno, graph):

    for i in range(antno):
        for j in range(len(colony[i]) - 1):
            current_node = colony[i][j]
            next_node = colony[i][j + 1]

            
            
#             print(tau[current_node][next_node])
            
#             print(current_node,next_node,1 / fitness(colony[i], graph))
            
            tau[current_node][next_node] = tau[current_node][next_node] + 1 / fitness(colony[i], graph)
            
#             print(tau[current_node][next_node])
#             print()
            
            tau[next_node][current_node] = tau[next_node][current_node] + 1 / fitness(colony[i], graph)

        
    return tau


In [12]:
def global_update_phromone(tau, graph, f_tour, global_rho):

    
    f_tour=f_tour-1
    
    print("***********")
    print(f_tour)
    
    l=len(f_tour)
    
    
    for i in range(l-1):
    
        current_node = f_tour[i]
        next_node = f_tour[i+1]
            
#         print(tau[current_node][next_node])
        
        a = tau[current_node][next_node]
        
        tau[current_node][next_node] += (1-global_rho) * 1 / fitness(f_tour, graph)
        
        b = tau[current_node][next_node] 
        
        print(b-a)
            
    return tau

In [13]:
## main ACO algo.

In [14]:
def aco_algo(graph, max_iter):
    
    max_iter = max_iter
    ant_no = 5
    n = len(graph)
    
    all_best_tour=[]
    
    tau0 = 10 * 1 / (len(graph) * np.mean(graph))  # initial phromone

    tau = tau0 * np.ones((n, n))  # phromone matx

    eta = 1 / graph  # desirability of each edge

    rho = 0.05  # evporation rate
    alpha = 1  # phromone exponential parameters
    beta = 1  # desirability exponential parameter

    best_fitness = np.inf

    global_rho = 0.5
    # main loop for ACO
    for i in range(max_iter):

        all_fitness = []

        colony = create_colony(graph, ant_no, tau, eta, alpha, beta)
        
        print(colony)

        for j in range(ant_no):
            all_fitness.append(fitness(colony[j], graph))


        min_tour = colony[np.argmin(all_fitness)]
        
        min_fitness = all_fitness[np.argmin(all_fitness)]

        if min_fitness < best_fitness:
            all_best_tour=[]
        
        if min_fitness <= best_fitness:
            best_fitness = min_fitness
            best_tour = colony[np.argmin(all_fitness)]
            best_tour = np.array(best_tour)+1
            
            
        if list(best_tour) not in all_best_tour:
            all_best_tour.append(list(best_tour))
        

        tau = local_update_phromone(tau, colony, ant_no, graph)
        
        
        
        
        tau = (1 - rho) * tau
        
        print(tau)
        
        tau = global_update_phromone(tau, graph, best_tour, global_rho)
        
        global_rho-=0.1
        
        print(tau)
        

        print(f'Iteration = {i + 1}, shortest_path ={best_fitness} , best_tour={best_tour}')
        
    print(f'\nShortest Path = {best_fitness}')
    
    print("All best tour = ",*all_best_tour)
    

In [15]:
graph

array([[0.        , 0.35693137, 1.06066017, 0.61008196],
       [0.35693137, 0.        , 0.78892332, 0.64899923],
       [1.06066017, 0.78892332, 0.        , 0.77278716],
       [0.61008196, 0.64899923, 0.77278716, 0.        ]])

In [16]:
start=time()
aco_algo(graph,4)
end=time()
print("\nTotal Time",end-start)

[[3, 1, 2, 0, 3], [1, 3, 0, 2, 1], [3, 1, 0, 2, 3], [0, 3, 2, 1, 0], [2, 1, 0, 3, 2]]
[[4.48284147 5.56878894 5.42861669 5.84540352]
 [5.56878894 4.48284147 5.84540352 5.42861669]
 [5.42861669 5.84540352 4.48284147 5.56878894]
 [5.84540352 5.42861669 5.56878894 4.48284147]]
***********
[0 3 2 1 0]
0.19772819728639401
0.19772819728639401
0.19772819728639401
0.19772819728639401
[[4.48284147 5.56878894 5.42861669 6.04313171]
 [5.76651714 4.48284147 5.84540352 5.42861669]
 [5.42861669 6.04313171 4.48284147 5.56878894]
 [5.84540352 5.42861669 5.76651714 4.48284147]]
Iteration = 1, shortest_path =2.5287238080453873 , best_tour=[1 4 3 2 1]
[[1, 3, 0, 2, 1], [3, 1, 0, 2, 3], [3, 0, 2, 1, 3], [1, 0, 3, 2, 1], [2, 1, 0, 3, 2]]
[[4.25869939 6.37629697 6.10296109 7.10353718]
 [6.56413876 4.25869939 6.91569539 6.10296109]
 [6.10296109 7.10353718 4.25869939 6.37629697]
 [6.91569539 6.10296109 6.56413876 4.25869939]]
***********
[2 1 0 3 2]
0.23727383674367264
0.23727383674367264
0.23727383674367264


## Travelling Salesman Problem

In [17]:
## kaggle  TSA Problem 
#  https://www.kaggle.com/tanmoyie/traveling-salesman-problem-to-optimize-travel

In [18]:
l = [[  0, 290, 250,  230,  190,  334, 365,   40], # Dhaka 1
    [290,   0, 337,  453,  396,  560, 581,  244], # Syhlet 2
    [250, 337,   0,  495,  396,  540, 120,  240], # Chittagonj 3
    [230, 453, 495,    0,  360,  150, 595,  242], # Rajshahi 4
    [190, 396, 396,  360,    0,  356, 496,  253], # Jossore 5
    [334, 560, 540,  150,  356,    0, 674,  275], # Dinajpur 6
    [365, 581, 120,  595,  496,  674,   0,  397], # Coxsbazar 7
    [40,  244, 240,  242,  253,  275, 397,    0]] # Narsingdi 8
    


l=np.array(l)



In [21]:
start=time()
aco_algo(l,1)  # aco_algo(graph, max_iteration) 
end=time()
print("\nTotal Time",end-start)

[[4, 0, 2, 7, 3, 5, 6, 1, 4], [7, 6, 0, 4, 2, 5, 3, 1, 7], [0, 7, 3, 6, 2, 5, 4, 1, 0], [3, 6, 7, 5, 1, 0, 4, 2, 3], [3, 1, 4, 5, 7, 0, 2, 6, 3]]
[[0.00385435 0.00451977 0.00458552 0.00385435 0.00484764 0.00385435
  0.0042017  0.004605  ]
 [0.00451977 0.00385435 0.00385435 0.00458399 0.00495388 0.00415141
  0.00420323 0.0042017 ]
 [0.00458552 0.00385435 0.00385435 0.00415141 0.00449876 0.00457006
  0.004605   0.00420323]
 [0.00385435 0.00458399 0.00415141 0.00385435 0.00385435 0.00455058
  0.00490206 0.00457159]
 [0.00484764 0.00495388 0.00449876 0.00385435 0.00385435 0.004605
  0.00385435 0.00385435]
 [0.00385435 0.00415141 0.00457006 0.00455058 0.004605   0.00385435
  0.00420323 0.0045337 ]
 [0.0042017  0.00420323 0.004605   0.00490206 0.00385435 0.00420323
  0.00385435 0.00449876]
 [0.004605   0.0042017  0.00420323 0.00457159 0.00385435 0.0045337
  0.00449876 0.00385435]]
***********
[3 1 4 5 7 0 2 6 3]
0.0002012072434607647
0.0002012072434607647
0.0002012072434607647
0.000201207243